# Use this notebook to update the recurrence flows tables in the viz database

## RUN THIS FIRST FOR ALL RECURRENCE CODE SECTIONS BELOW

In [9]:
# Change the sys path to pull from the parent folder for imports
import sys
sys.path.append('../')
from helper_functions.shared_functions import *

# Set the database user to the user with write access - get password from Secrets Manager if this doesn't work.
# Oct 2024, Moved the two enviro db user and password to the AWS_keys.py file. Not yet tested.

# Get a db_engine for writing back to the database.
viz_db = get_db_engine("viz")

### May 2 2024 - Phase 1 changes to the Recurrence Thresholds for CONUS - adjust HWT:
#### Reset Cluster 7 from 3.2 to 2.5  AND  Cluster 9 from 1.1 to 1.0
#### Set all HWT that are zero to minimum NWM v3 streamflow of 0.35 (cfs)

In [ ]:
# STEP 1: change the derived.huc8_rf_threshold table for clusters 7 & 9:

UPDATE derived.huc8_rf_thresholds
SET bankfull_rf = 'rf_2_5'
WHERE huc8_rf_thresholds.cluster = 7

UPDATE derived.huc8_rf_thresholds
SET bankfull_rf = 'rf_1_0'
WHERE huc8_rf_thresholds.cluster = 9

# STEP 2: make a backup copy of the derived.recurrence_flows_conus table

SELECT *
INTO derived.recurrence_flows_conus_backup
FROM derived.recurrence_flows_conus

# STEP 3: Add the runnoff cluster field into a new "dev" copy of the table

SELECT thresholds.*,
    clusters.cluster
INTO dev.recurrence_flows_conus
FROM derived.recurrence_flows_conus as thresholds
LEFT JOIN derived.featureid_huc_crosswalk as huc_xwalk ON huc_xwalk.feature_id = thresholds.feature_id
LEFT JOIN derived.huc8_rf_thresholds as clusters ON clusters.huc8 = huc_xwalk.huc8

# STEP 4: change the high_water_threshold values for all reaches of cluster 7 (from 3.2 to 2.5)

UPDATE dev.recurrence_flows_conus
SET high_water_threshold = rf_2_5
WHERE cluster = 7

# STEP 5: change the high_water_threshold values for all reaches of cluster 9 (from 1.1 to 1.0)

UPDATE dev.recurrence_flows_conus
SET high_water_threshold = rf_1_0
WHERE cluster = 9

# STEP 6: change all remaining zero high_water_threshold values to 0.35

UPDATE dev.recurrence_flows_conus
SET high_water_threshold = 0.35
WHERE high_water_threshold = 0

# STEP 7: copy the dev version of the recurrence_flows_conus table to the derived schema

DROP table derived.recurrence_flows_conus;
SELECT * 
INTO derived.recurrence_flows_conus
FROM dev.recurrence_flows_conus

## CONUS recurrence table

In [2]:
# Import the accompanying csv file to a dataframe - UPDATE THIS TO NEW CSV
new_recurrence_flows_df = pd.read_csv("recurrence_flows_conus_3.csv")

# Round all columns to two decimals
new_recurrence_flows_df = new_recurrence_flows_df.round(2)

In [7]:
# Define the database table you want to overwrite and upload the data - This can take 5-10 minutes due to the chunking required.
schema_name = "derived"
table_name = "recurrence_flows_conus"
new_recurrence_flows_df.to_sql(table_name, con=viz_db, schema=schema_name, if_exists='replace', index=False, chunksize=100000)

27734

In [8]:
# Change the feature_id column to integer type, and set the primary key to feature_id
run_sql_in_db(f"ALTER TABLE {schema_name}.{table_name} ALTER COLUMN feature_id TYPE integer;")
run_sql_in_db(f"ALTER TABLE {schema_name}.{table_name} ADD PRIMARY KEY (feature_id);")

This result object does not return rows. It has been closed automatically.
This result object does not return rows. It has been closed automatically.


## OTHER domain recurrence tables

In [3]:
# Import the accompanying csv file to a dataframe - UPDATE THIS TO NEW CSV
new_recurrence_flows_df = pd.read_csv("recurrence_flows_nwm_v3_AK.csv")

# Round all columns to two decimals
new_recurrence_flows_df = new_recurrence_flows_df.round(2)
new_recurrence_flows_df

,feature_id,rf_1_5,rf_2_0,rf_3_0,rf_4_0,rf_5_0,rf_10_0,rf_2_0_17c,rf_5_0_17c,rf_10_0_17c,rf_25_0_17c,rf_50_0_17c,high_water_threshold
0,19020190000001,0.00,0.00,0.35,0.35,0.35,0.35,0.00,0.00,0.00,0.00,0.00,0.00
1,19020190000002,0.35,0.35,0.35,0.35,0.35,0.71,0.36,0.47,0.58,0.76,0.94,0.36
2,19020190000003,0.35,0.35,4.59,4.94,5.30,6.11,1.65,5.14,9.05,16.26,23.49,1.65
3,19020190000004,0.00,0.35,0.35,0.35,0.35,0.35,0.00,0.00,0.00,0.00,0.00,0.00
4,19020190000005,8.83,9.53,11.30,12.01,12.78,15.01,9.82,13.03,15.69,19.71,23.24,9.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391517,75005400047360,0.00,0.00,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35
391518,75005400047361,0.00,0.00,0.00,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35
391519,75005400047362,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
391520,75005400047363,0.00,0.00,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35


In [4]:
# Define the database table you want to overwrite and upload the data - This can take 5-10 minutes due to the chunking required.
schema_name = "derived"
table_name = "recurrence_flows_ak"
new_recurrence_flows_df.to_sql(table_name, con=viz_db, schema=schema_name, if_exists='replace', index=False, chunksize=100000)

3522

In [7]:
# Change the feature_id column to integer type, and set the primary key to feature_id
run_sql_in_db(f"ALTER TABLE {schema_name}.{table_name} ALTER COLUMN feature_id TYPE integer;")
run_sql_in_db(f"ALTER TABLE {schema_name}.{table_name} ADD PRIMARY KEY (feature_id);")

This result object does not return rows. It has been closed automatically.


### Extra Stuff

In [ ]:
# This is how you save a local csv from an existing table - how I created the recurrence_flows_conus_2_1 csv.
import pandas as pd
df = pd.read_sql("SELECT * FROM derived.recurrence_flows_conus", con=viz_db, index_col="feature_id")
df.to_csv("recurrence_flows_conus_2_1.csv")

In [ ]:
# If you get a permissions error that you're not the owner of the table, a db admin like Shawn or Nick
# Can change the owner of the target table by running this SQL as postgres:
ALTER TABLE IF EXISTS derived.recurrence_flows_conus_test
    OWNER to viz_proc_admin_rw_user;

## SQL that creates a new table with HW threshold differences - to be used for a WPOD evaluation static service

In [ ]:
DROP TABLE IF EXISTS publish.streamflow_aep_high_water_comparison;
SELECT 
    row_number() over () as oid,
    current.feature_id,
    current.high_water_threshold as current_high_water_threshold,
    '3.0' as current_version,
    previous.high_water_threshold as previous_high_water_threshold,
    '2.1' as previous_version,
    ROUND((current.high_water_threshold - previous.high_water_threshold)::numeric, 2) as difference,
    CASE
        WHEN previous.high_water_threshold = 0
        THEN 0
        WHEN previous.high_water_threshold = -21474836.48
        THEN 0
        #ELSE ROUND(100 * ((current.high_water_threshold - previous.high_water_threshold) / current.high_water_threshold)::numeric, 0)
        # v1 = v21 or previous, and  v2 = v3 or current   ((v2 - v1)/v1)*100
        ELSE ROUND(100 * ((current.high_water_threshold - previous.high_water_threshold) / previous.high_water_threshold)::numeric, 0)
    END as percent_difference,
    strm_order,
    geom
INTO publish.streamflow_aep_high_water_comparison
FROM derived.recurrence_flows_conus AS current
LEFT JOIN derived.recurrence_flows_conus_2_1 AS previous
    ON previous.feature_id = current.feature_id
LEFT JOIN derived.channels_conus channel
    ON channel.feature_id = current.feature_id